In [1]:
# Setup
import numpy as np
import os
import symforce
from symforce import typing as T

symforce.set_symbolic_api("symengine")
symforce.set_log_level("warning")

In [2]:
symforce.set_epsilon_to_symbol()

In [3]:
from symforce import codegen
from symforce.codegen import codegen_util
from symforce import ops
import symforce.symbolic as sf
from symforce.values import Values
from symforce.notebook_util import display, display_code, display_code_file

In [4]:
# https://en.wikipedia.org/wiki/Double_integrator
q = sf.V3.symbolic("q")
display(q)

⎡q₀⎤
⎢  ⎥
⎢q₁⎥
⎢  ⎥
⎣q₂⎦

In [5]:
continuous_time_dynamics = sf.V2([q[1], q[2]])
display(continuous_time_dynamics)

⎡q₁⎤
⎢  ⎥
⎣q₂⎦

In [6]:
state = sf.V2([q[0], q[1]])


In [7]:
u = sf.V1(q[2])

In [8]:
# def double_integrator_discrete_time_dynamics(
#     x:sf.Vector2, u:sf.Vector1
# ) -> sf.Vector2:
#     A = continuous_time_dynamics.jacobian(x)
#     B = continuous_time_dynamics.jacobian(u)
#     qdot = sf.V2(A*x + B*u)
#     return qdot

In [9]:

# def rk4(x:sf.Vector2, u:sf.Vector1, dt:sf.Scalar) -> sf.Vector2: 
#     # vanilla RK4
#     k1 = dt * double_integrator_discrete_time_dynamics(x, u)
#     k2 = dt * double_integrator_discrete_time_dynamics(x + k1/2, u)
#     k3 = dt * double_integrator_discrete_time_dynamics(x + k2/2, u)
#     k4 = dt * double_integrator_discrete_time_dynamics(x + k3, u)
#     xplus1 =  (x + (1/6)*(k1 + 2*k2 + 2*k3 + k4))
#     return xplus1


Following is the get codegen the A and B dynamics by linearizing the Zdot

In [10]:
inputs_linearized_matrices = Values(
    state = sf.V2([q[0], q[1]]),
    u = sf.V1(q[2]),
)

In [11]:
A = continuous_time_dynamics.jacobian(state)
B = continuous_time_dynamics.jacobian(u)

In [12]:
outputs_linearized_matrices = Values(
    params = Values (
        A = A,
        B = B,
    )
)

In [13]:
linearized_matrices_codegen = codegen.Codegen(
    inputs=inputs_linearized_matrices,
    outputs=outputs_linearized_matrices,
    config=codegen.CppConfig(),
    name="linearized_matrices",
    return_key="params",
)

In [14]:
#call codegen
linearized_matrices_data = linearized_matrices_codegen.generate_function()

# Print what we generated
print("Files generated in {}:\n".format(linearized_matrices_data.output_dir))
for f in linearized_matrices_data.generated_files:
    print("  |- {}".format(os.path.relpath(f, linearized_matrices_data.output_dir)))

Files generated in /tmp/sf_codegen_linearized_matrices_pmo7f0m6:

  |- lcmtypes/linearized_matrices.lcm
  |- cpp/symforce/sym/linearized_matrices.h


Following is to codegen discrete time dynamics

In [15]:
inputs_discrete_time_dynamics = Values(
    state = sf.V2([q[0], q[1]]),
    u = sf.V1(q[2]),
    # A = A,
    # B = B,
)

In [16]:
outputs_discrete_time_dynamics = Values(
    state_dot = sf.V2(A*state + B*u)
    )

In [17]:
discrete_time_dynamics_codegen = codegen.Codegen(
    inputs=inputs_discrete_time_dynamics,
    outputs=outputs_discrete_time_dynamics,
    config=codegen.CppConfig(),
    name="double_integrator_discrete_time_dynamics",
    return_key="state_dot",
)


In [18]:
discrete_time_dynamics_data = discrete_time_dynamics_codegen.generate_function()

# Print what we generated
print("Files generated in {}:\n".format(discrete_time_dynamics_data.output_dir))
for f in discrete_time_dynamics_data.generated_files:
    print("  |- {}".format(os.path.relpath(f, discrete_time_dynamics_data.output_dir)))

Files generated in /tmp/sf_codegen_double_integrator_discrete_time_dynamics_dhzkj5f1:

  |- cpp/symforce/sym/double_integrator_discrete_time_dynamics.h


Following is the code gen for a quadratic cost function:

In [24]:
q_ref = sf.V3.symbolic("q_ref")


In [28]:
Q = sf.Matrix22.eye(),
R = sf.Scalar(0.0001),
# R = sf.V1("R"),

In [31]:
state_ref = sf.V2([q_ref[0], q_ref[1]])
u_ref = sf.V1(q_ref[2])

In [32]:
inputs_quad_cost = Values(
    state = sf.V2([q[0], q[1]]).T,
    u = sf.V1(q[2]),
    params = Values(
        Q = Q,
        R = R,
        state_ref = state_ref,
        u_ref = u_ref,
    )
)

In [42]:
Q = sf.M22.symbolic("Q"),
R = sf.V1.symbolic("R"),

temp = 0.5 * (state - state_ref).T * Q * (state - state_ref)






TypeError: can't multiply sequence by non-int of type 'symengine.lib.symengine_wrapper.MutableDenseMatrix'

In [50]:
display((state - state_ref).T * sf.M22(Q))

AssertionError: Gave args (([Q0_0, Q0_1]
[Q1_0, Q1_1]
,),) for <class 'symforce.geo.matrix.Matrix22'>

In [39]:
temp2 =  0.5 * (u - u_ref).T * R * (u - u_ref)

TypeError: can't multiply sequence by non-int of type 'symengine.lib.symengine_wrapper.MutableDenseMatrix'

In [33]:
outputs_quad_cost = Values(
    cost = (0.5 * (state - state_ref).T * Q * (state - state_ref) + 0.5 * (u - u_ref).T * R * (u - u_ref))
    )

TypeError: can't multiply sequence by non-int of type 'symengine.lib.symengine_wrapper.MutableDenseMatrix'

In [21]:
display(inputs_quad_cost)
display(outputs_quad_cost)
display(Q)
display(R)

NameError: name 'inputs_quad_cost' is not defined

In [ ]:
quad_cost_codegen = codegen.Codegen(
    inputs=inputs_quad_cost,
    outputs=outputs_quad_cost,
    config=codegen.CppConfig(),
    name="quadratic_cost",
    return_key="cost",
)


In [ ]:
quad_cost_data = quad_cost_codegen.generate_function()

# Print what we generated
print("Files generated in {}:\n".format(quad_cost_data.output_dir))
for f in quad_cost_data.generated_files:
    print("  |- {}".format(os.path.relpath(f, quad_cost_data.output_dir)))